In [86]:
import pandas as pd
import glob
import time
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime, timedelta
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)
from plotly.graph_objs import *
#from pyzipcode import ZipCodeDatabase
#from geopy import geocoders
from geopy.geocoders import Nominatim

In [87]:
geolocator = Nominatim()
location = geolocator.geocode("94306")
location.latitude



37.4233552

In [88]:
# READ IN AND MERGE CAMP AND STUDENT FILES

stud = pd.read_csv('/home/harisk87/Dropbox/PythonProjects/EduCare/StudentCA.csv', parse_dates = ['enroll_date', 'end_date', 'date_of_birth'])
camps = pd.read_csv('/home/harisk87/Dropbox/PythonProjects/EduCare/MathCampAll.csv', parse_dates = ['open_date'])

students = pd.merge(stud, camps, on='camp_id')
students = students.rename(columns={'city_x': 'city_student', 'country': 'country_student', 'zip_code_x': 'zip_student', 'state_code_x': 'state_student', 'city_y':'city_camp', 'zip_code_y':'zip_camp', 'state_code_y':'state_camp,', 'country_code':'country_camp'})

students.head()

,camp_id,student_id,gender,date_of_birth,enroll_date,end_date,grade_level,city_student,country_student,zip_student,state_student,city_camp,zip_camp,"state_camp,",country_camp,open_date,center_days
0,2824,61913,M,2002-01-22,2015-09-01,2015-11-30,12,Santa Rosa,USA,95409,CA,Windsor,95492,CA,US,2015-01-01,"[2, 5]"
1,2824,61922,F,2002-04-28,2015-01-01,2016-09-18,11,Santa Rosa,USA,95403,CA,Windsor,95492,CA,US,2015-01-01,"[2, 5]"
2,2824,61938,M,2000-12-13,2015-01-01,2015-08-30,12,Santa Rosa,USA,95404,CA,Windsor,95492,CA,US,2015-01-01,"[2, 5]"
3,2824,61958,M,2008-07-21,2016-01-05,2016-09-18,6,Windsor,USA,95492,CA,Windsor,95492,CA,US,2015-01-01,"[2, 5]"
4,2824,61981,F,2008-01-21,2016-03-04,2016-05-28,7,Santa Rosa,USA,95409,CA,Windsor,95492,CA,US,2015-01-01,"[2, 5]"


In [89]:
# REMOVE NULLS AND DUPLICATE DATA
print(str(len(students)) + ' Total Students Enrolled')

#print(students.isnull().sum())
# 215 students who did not submit worksheets - dont know if they quit on the first day 
# or are enrolled and did not submit worksheets.. leave them out for now

print(str(len(students.dropna(how='any'))) + ' remaining students after nulls dropped ~5%')
students = students.dropna(how='any')

print(len(students))
print(str(len(students[students.duplicated()]))+' absolute duplicates in the data')
#print(str(len(students[students.duplicated(subset=['student_id'])])))
# no duplicates in data, very clean

4428 Total Students Enrolled
4204 remaining students after nulls dropped ~5%
4204
0 absolute duplicates in the data


In [90]:
# TIME ENROLLED FOR EACH STUDENT
td = (students['end_date'] - students['enroll_date'])
students["time_enrolled"] = (td / np.timedelta64(1, 'D')).astype(int)
#students.head()

In [91]:
#dealing with end dates of those who are still enrolled?
#to make it easy maybe eliminate students who are still enrolled? 
# and do long-term vs short-term enrollment


#last day they pulled data will depend on the center??
#probably need to get last day they pulled data from each camp? 
#- for now ill just look at 1 week bfore

dy = 7
t = students[students['end_date'] <= students.end_date.max()-timedelta(days=dy)]['time_enrolled']
trace1 = Histogram(x=t)
data = [trace1]
layout = Layout(xaxis=dict(title = 'Time of Enrollment'),
                 yaxis=dict(title='Number of students'),
                title='Time of enrollemnt for students')


fig1 = Figure(data=data, layout=layout)
iplot(fig1)

#iplot([Histogram(x=t)])

students = students[students['end_date'] <= students.end_date.max()-timedelta(days=dy)]
len(students)

2679

In [92]:
#len(students[students['time_enrolled'] < 240])

In [93]:
students['time_enrolled'].median()

231.0

In [94]:
# PICK A BOUNDARY - LESS THAN THAT IS SHORT-TERM, MORE IS LONG-TERM

# WHat is the right boundary to define?


ndays = 230#120#400#240 #120
students['short_term'] = 0
students.ix[students['time_enrolled'] <= ndays, 'short_term'] = 1


In [95]:
# First Feature: Gender - classify by gender - male or female, deal with unclean data
# there re entries in the gender column called: {u'@xml:space': u'preserve'} 
#- i have no idea if these are male or female, i think they were not filled out on the form 
# so they are left as blank - I will eliminate these

grouped = students.groupby('gender')
students = students.drop(grouped.get_group("{u'@xml:space': u'preserve'}").index) 
len(students)         

2672

In [96]:
# AGE

# there are students who are 50 plus years old.. is this correct data?
# are there too many features and should i group the features? - 28 features including the oldies - made it into one continuous feature
#  i can eliminate these students, to have fewer features, 
# but actually for now maybe include all then later eliminate to improve algorithm

ag_2016 = students.end_date.max() - students['date_of_birth']
ag_enroll = students['enroll_date'] - students['date_of_birth']
ag_end = students['end_date'] - students['date_of_birth']
students['age_2016'] = (((ag_2016 / np.timedelta64(1, 'D')).astype(int))/365).astype(int)
students['age_enroll'] = (((ag_enroll / np.timedelta64(1, 'D')).astype(int))/365).astype(int)
students['age_end'] = (((ag_end / np.timedelta64(1, 'D')).astype(int))/365).astype(int)

print(students.groupby('age_2016').size())
print(students.groupby('age_enroll').size())
print(students.groupby('age_end').size())

#print(len(students.groupby('age').size()))
#students[students['age'] == 54]

age_2016
2       2
3       7
4      17
5      42
6      74
7     127
8     165
9     225
10    250
11    286
12    301
13    293
14    206
15    203
16    150
17    102
18     88
19     54
20     31
21     24
22     10
23      4
24      5
29      1
48      1
50      2
51      1
54      1
dtype: int64
age_enroll
2       5
3      48
4     114
5     211
6     273
7     341
8     333
9     328
10    310
11    253
12    199
13    113
14     69
15     41
16     17
17      8
18      2
19      1
26      1
42      2
46      3
dtype: int64
age_end
2       3
3      20
4      45
5     120
6     191
7     276
8     281
9     345
10    333
11    309
12    271
13    207
14    143
15     60
16     37
17     19
18      5
20      1
27      1
42      2
46      2
47      1
dtype: int64


In [97]:
# GRADE LEVEL

# do i group students with these 15 features - 1 through 15, 
# or do i split them up into elementary, middle and high school
# what do grade levels 1 through 21 mean
# these are probably way too many features - leave it be for now
grlv = students['grade_level']
students.groupby('grade_level').size()


grade_level
1      10
2      23
3      25
4      40
5     117
6     197
7     271
8     333
9     398
10    356
11    276
12    150
13    164
14    119
15     86
16     62
17     22
18     11
19      4
20      3
21      5
dtype: int64

In [98]:
# NUMBER OF DAYS CAMP IS OPEN
# students['center_days'].count
# divide by 3 not subtract 4
students['days_open'] = (students['center_days'].str.len()/3).astype(int)
students.groupby('days_open').size()

days_open
2    1923
3     749
dtype: int64

In [99]:
# CITY CAMP IS IN
# looks good and clean


students.groupby('city_camp').size()

city_camp
Concord          121
Fairfield        107
Fresno            46
Los Alamitos     626
Mission Viejo    234
San Clemente      27
San Francisco    103
San Jose         221
San Rafael       354
Santa Clara       53
Stockton         410
Torrance          71
Victorville      113
Whittier         100
Windsor           86
dtype: int64

In [100]:
# CITY STUDENT IS IN - 
# maybe leave this out for now cuz there are 194 features..

len(students.groupby('city_student').size())

students.groupby('zip_camp').size()


zip_camp
90505     71
90602    100
90720    626
92395    113
92673     27
92691    234
93722     46
94123    103
94519    121
94534    107
94901    354
95054     53
95131    221
95207    410
95492     86
dtype: int64

In [101]:
# engineering features - distance and census data

students.groupby('zip_student').size()
students = students[students['zip_student'].str.startswith("9")]
students['dist'] = ""

In [118]:
#students = pd.read_pickle('students')
students.loc[211]#.head(210)

camp_id                           2825
student_id                      480561
gender                               F
date_of_birth      2002-11-06 00:00:00
enroll_date        2011-07-07 00:00:00
end_date           2012-10-29 00:00:00
grade_level                         10
city_student                SAN RAFAEL
country_student                    USA
zip_student                      94901
state_student                       CA
city_camp                   San Rafael
zip_camp                         94901
state_camp,                         CA
country_camp                        US
open_date          2010-12-01 00:00:00
center_days                  [2, 4, 6]
time_enrolled                      480
short_term                           0
age_2016                            13
age_enroll                           8
age_end                              9
days_open                            3
dist                                 0
Name: 211, dtype: object

In [196]:
geolocator = Nominatim()
from geopy.distance import vincenty
from geopy.distance import great_circle
for a in range(2501,len(students)):
    if students['zip_student'].iloc[a] == students['zip_camp'].iloc[a]:
        students['dist'].iloc[a] = 0
    else:
        locationa = geolocator.geocode(students['zip_camp'].iloc[a], timeout=None)
        locationb = geolocator.geocode(students['zip_student'].iloc[a], timeout=None)
        if locationa and locationb:
            zipa = (locationa.latitude, locationa.longitude)
            zipb = (locationb.latitude, locationb.longitude)
            students['dist'].iloc[a] = great_circle(zipa, zipb).miles
    #students['camp_lon'].iloc[a] = (geolocator.geocode(students['zip_camp'].iloc[a], timeout=None)).longitude
#students_all = pd.read_pickle('students')
#print(students.head(300))
students.to_pickle('students') 
#students.head(50)

In [383]:
#students = pd.read_pickle('students')

# TO-DOs:
# look for empty rows 
studentsshort = students[students['dist'] != ""]#[['zip_student','zip_camp', 'dist']]
len(studentsshort)
# look for rows with distance = 0  -to check - all look fine
#students[students['dist'] == 0][['zip_student','zip_camp', 'dist']]

2611

In [397]:
studentsshort = students[students['dist'] != ""]#[['zip_student','zip_camp', 'dist']]
len(studentsshort)
studentsshortnew = studentsshort[studentsshort['dist']<100]
len(studentsshortnew)
studentsshortnew['dist'] = studentsshortnew['dist'] / studentsshortnew['dist'].max()
studentsshortnew['dist']

/home/harisk87/anaconda2/envs/my_projects_env/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



0        0.102496
2       0.0488616
4        0.102496
5        0.102496
9        0.184793
10              0
11              0
12              0
13              0
14       0.102496
15      0.0488616
16      0.0763632
17       0.102496
18      0.0488616
19      0.0488616
20              0
21              0
23      0.0763632
24       0.142875
26      0.0220801
30              0
33       0.102496
34              0
36              0
37      0.0220801
41              0
42              0
45        0.54008
48       0.142875
49       0.142875
          ...    
4362    0.0787015
4364    0.0604966
4366            0
4367            0
4370    0.0604966
4372     0.311619
4373            0
4376    0.0604966
4382            0
4385            0
4386    0.0787015
4390            0
4393    0.0787015
4395            0
4397    0.0787015
4401    0.0604966
4402    0.0787015
4404            0
4405            0
4406    0.0787015
4407            0
4408     0.303296
4410    0.0787015
4411            0
4412     0

In [258]:
len(studentsshortnew)


2235

In [398]:
#load census data

income1 = pd.read_csv('/home/harisk87/Dropbox/PythonProjects/Insight-Project/IncomeData.csv')
income = income1[['GEO.id2', 'HC02_EST_VC02', 'HC02_EST_VC04']]
income = income.rename(columns={'GEO.id2':'zip_student'})
# get zip codes and median income
studentsshortnew = pd.merge(studentsshortnew, income, on='zip_student')
# put in median income into clumns
studentsshortnew = studentsshortnew.rename(columns={'HC02_EST_VC02': 'household_income', 'HC02_EST_VC04': 'median_income'})


In [233]:
# look for rows with weird distances 

trace1 = Histogram(x=studentsshort['dist'])
data = [trace1]
layout = Layout(xaxis=dict(title = 'Distance(miles)'),yaxis=dict(title='Number of students'),title='Distance Distribution')


fig1 = Figure(data=data, layout=layout)
iplot(fig1)

studentsshort[studentsshort['dist'] > 1000][['zip_student','zip_camp', 'dist']]








,zip_student,zip_camp,dist
22,95448,95492,5727.79
32,95448,95492,5727.79
38,95448,95492,5727.79
56,95448,95492,5727.79
59,95448,95492,5727.79
60,95448,95492,5727.79
231,94547,94901,5562.79
236,94547,94901,5562.79
757,94912,94901,6004.61
1072,96534,94534,6476.59


In [197]:
# see ow many rows are left
# remove data and run LR - and see accuracy is correct
# add distance and camp city and LR
# use only distance and LR

,camp_id,student_id,gender,date_of_birth,enroll_date,end_date,grade_level,city_student,country_student,zip_student,...,country_camp,open_date,center_days,time_enrolled,short_term,age_2016,age_enroll,age_end,days_open,dist
4413,3221,555476,M,2008-08-22,2015-07-10,2015-07-28,5,FRESNO,USA,93722,...,US,2014-09-01,"[4, 6]",18,1,8,6,6,2,0
4414,3221,555518,M,2009-08-26,2015-04-22,2015-05-06,4,FRESNO,USA,93723,...,US,2014-09-01,"[4, 6]",14,1,7,5,5,2,3.96491
4417,3221,555993,F,2005-07-11,2015-05-16,2015-05-19,7,MADERA,USA,93638,...,US,2014-09-01,"[4, 6]",3,1,11,9,9,2,15.8741
4421,3221,556263,F,2003-09-29,2014-09-19,2016-02-25,13,FRESNO,USA,93722,...,US,2014-09-01,"[4, 6]",524,0,12,10,12,2,0
4426,3221,556873,F,2002-03-14,2014-09-19,2015-03-10,14,FRESNO,USA,93722,...,US,2014-09-01,"[4, 6]",172,1,14,12,12,2,0


In [399]:
# CENSUS DATA:
# open new notebook
#load census data
# get zip codes and median income
# put in median income into clumns[
#type(studentsshortnew['median_income'][0])
#int(studentsshortnew['household_income'])#.astype(int)
#studentsshortnew[studentsshortnew['median_income']=='']
#type(studentsshortnew['median_income'].values)#[0]
studentsshortnew['median_income'] = pd.to_numeric(studentsshortnew['median_income'], errors='coerce')
studentsshortnew['household_income'] = pd.to_numeric(studentsshortnew['household_income'], errors='coerce')

studentsshortnew = studentsshortnew.dropna(how='any')
#print(len(studentsshortnew))

#print (studentsshortnew[ pd.to_numeric(studentsshortnew['median_income'], errors='coerce').isnull()])
#studentsshortnew[studentsshortnew['household_income'].isnull()]
#studentsshortnew[studentsshortnew['median_income'] == NaN]
#studentsshortnew.isnull
#print(str(len(studentsshortnew.dropna(how='any'))))
#print(str(len(students.dropna(how='any'))) + ' remaining students after nulls dropped ~5%')
#students = students.dropna(how='any')

studentsshortnew['median_income'] = studentsshortnew['median_income']/studentsshortnew['median_income'].max()
studentsshortnew['household_income'] = studentsshortnew['household_income']/studentsshortnew['household_income'].max()
#studentsshortnew['age_enroll'] = studentsshortnew['age_enroll']/studentsshortnew['age_enroll'].max()
#studentsshortnew['grade_level'] = studentsshortnew['grade_level']/studentsshortnew['grade_level'].max()
#studentsshortnew['days_open'] = studentsshortnew['days_open']/studentsshortnew['days_open'].max()

In [400]:
studentsshortnew.head()

,camp_id,student_id,gender,date_of_birth,enroll_date,end_date,grade_level,city_student,country_student,zip_student,...,center_days,time_enrolled,short_term,age_2016,age_enroll,age_end,days_open,dist,household_income,median_income
0,2824,61913,M,2002-01-22,2015-09-01,2015-11-30,12,Santa Rosa,USA,95409,...,"[2, 5]",90,1,14,13,13,2,0.102496,0.298022,0.30956
1,2824,61981,F,2008-01-21,2016-03-04,2016-05-28,7,Santa Rosa,USA,95409,...,"[2, 5]",85,1,8,8,8,2,0.102496,0.298022,0.30956
2,2824,61991,F,2004-09-25,2016-03-04,2016-05-26,7,Santa Rosa,USA,95409,...,"[2, 5]",83,1,11,11,11,2,0.102496,0.298022,0.30956
3,2824,62145,F,2005-06-11,2015-01-01,2015-08-23,10,Santa Rosa,USA,95409,...,"[2, 5]",234,0,11,9,10,2,0.102496,0.298022,0.30956
4,2824,62186,F,2007-09-28,2015-05-01,2015-05-24,6,Santa Rosa,USA,95409,...,"[2, 5]",23,1,8,7,7,2,0.102496,0.298022,0.30956


In [290]:
#from sklearn import preprocessing
#min_max_scaler = preprocessing.MinMaxScaler()
#X_train_minmax = min_max_scaler.fit_transform(xprev)
#X_train_minmax

array([[ 0.25      ,  0.55      ,  0.        ],
       [ 0.27272727,  0.55      ,  0.        ],
       [ 0.13636364,  0.3       ,  0.        ],
       ..., 
       [ 0.15909091,  0.3       ,  0.        ],
       [ 0.18181818,  0.6       ,  0.        ],
       [ 0.22727273,  0.65      ,  0.        ]])

In [409]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn import linear_model, datasets, metrics
from sklearn.cross_validation import train_test_split

logreg = LogisticRegression(penalty = "l2", C=0.1, class_weight = 'balanced', tol = 0.0001)
#logreg = linear_model.LogisticRegression()
feature_cols = ['age_enroll','days_open', 'grade_level']

xprev = studentsshortnew[feature_cols]#.reset_index()[feature_cols]
y = studentsshortnew['short_term']#.reset_index()['short_term']x
x1 = []
x1 = pd.concat([xprev, pd.get_dummies(studentsshortnew['gender'])],axis=1)
x = []
x = pd.concat([x1, pd.get_dummies(studentsshortnew['city_camp'])],axis=1)


X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2)
L = logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)

print 'Accuracy of model on test set is ' + str(metrics.accuracy_score(y_test,y_pred)*100) +' %'
print('F1 scores are long-term/short-term' + str(f1_score(y_test, y_pred, average=None)))
#print('Weighted F1 score is ' + str(f1_score(y_test, y_pred, average='weighted')))
print('Precision scores are lt-st' + str(precision_score(y_test, y_pred, average=None)))
#print('Weighted Precision score is ' + str(precision_score(y_test, y_pred, average='weighted')))
print('Recall scores are lt-st' + str(recall_score(y_test, y_pred, average=None)))
#print('Weighted Recall score is ' + str(recall_score(y_test, y_pred, average='weighted')))
print metrics.confusion_matrix(y_test, y_pred)
print ("Predicting everything as falling as long term will give accuracy of "+ str((len([i for i in y_test if i == 0])/float(len(y_test)))*100))

fimplr = np.abs(logreg.coef_)
vv = list(x)
#ll = np.argsort(np.abs(logreg.coef_))
#print(fimplr.ravel())
#print(vv)
#print(ll)
#print(ll[vv])


data = [Bar(x=vv,y=fimplr.ravel())]

layout = Layout(xaxis=dict(title = 'Features'),
                 yaxis=dict(title='Coefficients'),
                title='Feature Coefficients')


fig1 = Figure(data=data, layout=layout)
iplot(fig1)


Accuracy of model on test set is 80.9843400447 %
F1 scores are long-term/short-term[ 0.8098434  0.8098434]
Precision scores are lt-st[ 0.81531532  0.80444444]
Recall scores are lt-st[ 0.80444444  0.81531532]
[[181  44]
 [ 41 181]]
Predicting everything as falling as long term will give accuracy of 50.3355704698


In [471]:
scores = pd.read_csv('/home/harisk87/Dropbox/PythonProjects/EduCare/ScoresCA.csv')
scores2 = scores.groupby(['student_id'], sort=False)['grade_level'].min()
scores4 = pd.DataFrame({'student_id':scores2.keys(), 'grade_level_begin':scores2.values})
studentsn = pd.merge(studentsshortnew, scores4, on='student_id')
studentsn['skill_increase'] = ((studentsn['grade_level']-studentsn['grade_level_begin'])/studentsn['time_enrolled'])
studentsn = studentsn.dropna(how='any')

In [488]:
studentsn = studentsn.replace([np.inf, -np.inf], np.nan).dropna(subset=["skill_increase"], how="all")
studentsn[np.isinf(studentsn['skill_increase'])]
studentsn['skill_increase'] = studentsn['skill_increase']/studentsn['skill_increase'].max()

In [610]:
#studentsn[np.isinf(studentsn['skill_increase'])]['skill_increase']=0
#students[studentsn['skill_increase'] == np.inf]#.isinf().sum()
#studentsn[np.isinf(studentsn['skill_increase'])]
#studentsn['skill_increase'].replace(np.inf, 0)
studentsn.head()

,camp_id,student_id,gender,date_of_birth,enroll_date,end_date,grade_level,city_student,country_student,zip_student,...,age_2016,age_enroll,age_end,days_open,dist,household_income,median_income,grade_level_begin,skill_increase,shorter_term
0,2824,61913,M,2002-01-22,2015-09-01,2015-11-30,12,Santa Rosa,USA,95409,...,14,13,13,2,0.102496,0.298022,0.30956,11,0.008333,1
1,2824,61981,F,2008-01-21,2016-03-04,2016-05-28,7,Santa Rosa,USA,95409,...,8,8,8,2,0.102496,0.298022,0.30956,5,0.017647,1
2,2824,61991,F,2004-09-25,2016-03-04,2016-05-26,7,Santa Rosa,USA,95409,...,11,11,11,2,0.102496,0.298022,0.30956,6,0.009036,1
3,2824,62145,F,2005-06-11,2015-01-01,2015-08-23,10,Santa Rosa,USA,95409,...,11,9,10,2,0.102496,0.298022,0.30956,2,0.025641,0
4,2824,62186,F,2007-09-28,2015-05-01,2015-05-24,6,Santa Rosa,USA,95409,...,8,7,7,2,0.102496,0.298022,0.30956,5,0.032609,1


In [644]:
logreg = LogisticRegression(penalty = "l2", C=1000000, class_weight = 'balanced', tol = 0.0001)
feature_cols = ['age_enroll','days_open', 'dist', 'median_income',  'grade_level','grade_level_begin', 'skill_increase']

xprev = studentsn[feature_cols]#.reset_index()[feature_cols]
y = studentsn['short_term']#.reset_index()['short_term']x
x = xprev

X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2)
L = logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)

print 'Accuracy of model on test set is ' + str(metrics.accuracy_score(y_test,y_pred)*100) +' %'
print('F1 scores are long-term/short-term' + str(f1_score(y_test, y_pred, average=None)))
print('Precision scores are lt-st' + str(precision_score(y_test, y_pred, average=None)))
print('Recall scores are lt-st' + str(recall_score(y_test, y_pred, average=None)))
print metrics.confusion_matrix(y_test, y_pred)
print ("Predicting everything as falling as long term will give accuracy of "+ str((len([i for i in y_test if i == 0])/float(len(y_test)))*100))

fimplr = logreg.coef_
vv = list(x)


data = [Bar(x=vv,y=fimplr.ravel())]

layout = Layout(xaxis=dict(title = 'Features'),
                 yaxis=dict(title='Coefficients'),
                title='Feature Coefficients')


fig1 = Figure(data=data, layout=layout)
iplot(fig1)


Accuracy of model on test set is 96.4125560538 %
F1 scores are long-term/short-term[ 0.96428571  0.96396396]
Precision scores are lt-st[ 0.98630137  0.94273128]
Recall scores are lt-st[ 0.94323144  0.98617512]
[[216  13]
 [  3 214]]
Predicting everything as falling as long term will give accuracy of 51.3452914798


In [605]:
stsk = studentsn[studentsn['short_term']==1]['skill_increase']
ltsk = studentsn[studentsn['short_term']==0]['skill_increase']
print("The p value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).pvalue/float(2)) + " and the t value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).statistic/float(2)))
print ("The effect size is " + str(cohensd(stsk, ltsk)))
print("SHort terms increase skills on average at rate " + str(np.mean(stsk)))
print("LOng terms increase skills on avergae at rate " + str(np.mean(ltsk)))
#print("Females are enrolled on average for a longer number of days, so might want to target males, but this is probably not a high return on investment because the distributions are not significantly different, and the effect on predicting churn is also small.")
stsk = studentsn[studentsn['short_term']==1]['grade_level_begin']
ltsk = studentsn[studentsn['short_term']==0]['grade_level_begin']
print("\n The p value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).pvalue/float(2)) + " and the t value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).statistic/float(2)))
print ("The effect size is " + str(cohensd(stsk, ltsk)))
print("SHort terms begin at grade level " + str(np.mean(stsk)))
print("LOng terms begin at grade level " + str(np.mean(ltsk)))
#print("Females are enrolled on average for a longer number of days, so might want to target males, but this is probably not a high return on investment because the distributions are not significantly different, and the effect on predicting churn is also small.")
stsk = studentsn[studentsn['short_term']==1]['dist']
ltsk = studentsn[studentsn['short_term']==0]['dist']
print("\n The p value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).pvalue/float(2)) + " and the t value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).statistic/float(2)))
print ("The effect size is " + str(cohensd(stsk, ltsk)))
print("SHort terms commute " + str(np.mean(stsk)))
print("LOng terms commute " + str(np.mean(ltsk)))
#print("Females are enrolled on average for a longer number of days, so might want to target males, but this is probably not a high return on investment because the distributions are not significantly different, and the effect on predicting churn is also small.")
stsk = studentsn[studentsn['short_term']==1]['median_income']
ltsk = studentsn[studentsn['short_term']==0]['median_income']
print("\nThe p value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).pvalue/float(2)) + " and the t value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).statistic/float(2)))
print ("The effect size is " + str(cohensd(stsk, ltsk)))
print("SHort terms income " + str(np.mean(stsk)))
print("LOng terms income " + str(np.mean(ltsk)))
#print("Females are enrolled on average for a longer number of days, so might want to target males, but this is probably not a high return on investment because the distributions are not significantly different, and the effect on predicting churn is also small.")
stsk = studentsn[studentsn['short_term']==1]['age_enroll']
ltsk = studentsn[studentsn['short_term']==0]['age_enroll']
print("\nThe p value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).pvalue/float(2)) + " and the t value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).statistic/float(2)))
print ("The effect size is " + str(cohensd(stsk, ltsk)))
print("SHort terms enroll age " + str(np.mean(stsk)))
print("LOng terms enroll age " + str(np.mean(ltsk)))
#print("Females are enrolled on average for a longer number of days, so might want to target males, but this is probably not a high return on investment because the distributions are not significantly different, and the effect on predicting churn is also small.")
stsk = studentsn[studentsn['short_term']==1]['grade_level']
ltsk = studentsn[studentsn['short_term']==0]['grade_level']
print("\nThe p value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).pvalue/float(2)) + " and the t value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).statistic/float(2)))
print ("The effect size is " + str(cohensd(stsk, ltsk)))
print("SHort terms exit level " + str(np.mean(stsk)))
print("LOng terms exit level " + str(np.mean(ltsk)))
#print("Females are enrolled on average for a longer number of days, so might want to target males, but this is probably not a high return on investment because the distributions are not significantly different, and the effect on predicting churn is also small.")
stsk = studentsn[studentsn['short_term']==1]['days_open']
ltsk = studentsn[studentsn['short_term']==0]['days_open']
print("\nThe p value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).pvalue/float(2)) + " and the t value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).statistic/float(2)))
print ("The effect size is " + str(cohensd(stsk, ltsk)))
print("SHort terms days open " + str(np.mean(stsk)))
print("LOng terms days open " + str(np.mean(ltsk)))

The p value is 3.02873596065e-27 and the t value is 5.5139276208
The effect size is 0.465218490351
SHort terms increase skills on average at rate 0.0219624424506
LOng terms increase skills on avergae at rate 0.00695645365014

 The p value is 0.0127760178808 and the t value is 1.11723781798
The effect size is 0.0947476616482
SHort terms begin at grade level 6.14133333333
LOng terms begin at grade level 5.90760869565

 The p value is 0.310625146601 and the t value is 0.247074074105
The effect size is 0.0209420240341
SHort terms commute 0.0617287724045
LOng terms commute 0.0602362976335

The p value is 0.422897904187 and the t value is 0.0972540495816
The effect size is 0.00824202570696
SHort terms income 0.419150221348
LOng terms income 0.4180760078

The p value is 7.01374482192e-11 and the t value is 3.22366082497
The effect size is 0.272972052696
SHort terms enroll age 9.15466666667
LOng terms enroll age 8.25996376812

The p value is 5.0484840687e-86 and the t value is -10.2570393341
T

In [628]:
studentsn['shorter_term'] = 0
studentsn.ix[studentsn['time_enrolled'] <= 120, 'shorter_term'] = 1

logreg = LogisticRegression(penalty = "l2", C=0.1, class_weight = 'balanced', tol = 0.0001)
feature_cols = ['age_enroll','days_open', 'dist', 'median_income',  'grade_level','grade_level_begin', 'skill_increase']

xprev = studentsn[feature_cols]#.reset_index()[feature_cols]
y = studentsn['shorter_term']#.reset_index()['short_term']x
x = xprev

X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.8)
L = logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)

print 'Accuracy of model on test set is ' + str(metrics.accuracy_score(y_test,y_pred)*100) +' %'
print('F1 scores are long-term/short-term' + str(f1_score(y_test, y_pred, average=None)))
print('Precision scores are lt-st' + str(precision_score(y_test, y_pred, average=None)))
print('Recall scores are lt-st' + str(recall_score(y_test, y_pred, average=None)))
print metrics.confusion_matrix(y_test, y_pred)
print ("Predicting everything as falling as long term will give accuracy of "+ str((len([i for i in y_test if i == 0])/float(len(y_test)))*100))

fimplr = logreg.coef_
vv = list(x)


data = [Bar(x=vv,y=fimplr.ravel())]

layout = Layout(xaxis=dict(title = 'Features'),
                 yaxis=dict(title='Coefficients'),
                title='Feature Coefficients')


fig1 = Figure(data=data, layout=layout)
iplot(fig1)


Accuracy of model on test set is 81.3340807175 %
F1 scores are long-term/short-term[ 0.85955293  0.72180451]
Precision scores are lt-st[ 0.89385965  0.67080745]
Recall scores are lt-st[ 0.82778229  0.78119349]
[[1019  212]
 [ 121  432]]
Predicting everything as falling as long term will give accuracy of 69.0022421525


In [604]:
stsk = studentsn[studentsn['shorter_term']==1]['skill_increase']
ltsk = studentsn[studentsn['shorter_term']==0]['skill_increase']
print("The p value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).pvalue/float(2)) + " and the t value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).statistic/float(2)))
print ("The effect size is " + str(cohensd(stsk, ltsk)))
print("SHort terms increase skills on average at rate " + str(np.mean(stsk)))
print("LOng terms increase skills on avergae at rate " + str(np.mean(ltsk)))
#print("Females are enrolled on average for a longer number of days, so might want to target males, but this is probably not a high return on investment because the distributions are not significantly different, and the effect on predicting churn is also small.")
stsk = studentsn[studentsn['shorter_term']==1]['grade_level_begin']
ltsk = studentsn[studentsn['shorter_term']==0]['grade_level_begin']
print("\n The p value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).pvalue/float(2)) + " and the t value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).statistic/float(2)))
print ("The effect size is " + str(cohensd(stsk, ltsk)))
print("SHort terms begin at grade level " + str(np.mean(stsk)))
print("LOng terms begin at grade level " + str(np.mean(ltsk)))
#print("Females are enrolled on average for a longer number of days, so might want to target males, but this is probably not a high return on investment because the distributions are not significantly different, and the effect on predicting churn is also small.")
stsk = studentsn[studentsn['shorter_term']==1]['dist']
ltsk = studentsn[studentsn['shorter_term']==0]['dist']
print("\n The p value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).pvalue/float(2)) + " and the t value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).statistic/float(2)))
print ("The effect size is " + str(cohensd(stsk, ltsk)))
print("SHort terms commute " + str(np.mean(stsk)))
print("LOng terms commute " + str(np.mean(ltsk)))
#print("Females are enrolled on average for a longer number of days, so might want to target males, but this is probably not a high return on investment because the distributions are not significantly different, and the effect on predicting churn is also small.")
stsk = studentsn[studentsn['shorter_term']==1]['median_income']
ltsk = studentsn[studentsn['shorter_term']==0]['median_income']
print("\nThe p value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).pvalue/float(2)) + " and the t value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).statistic/float(2)))
print ("The effect size is " + str(cohensd(stsk, ltsk)))
print("SHort terms income " + str(np.mean(stsk)))
print("LOng terms income " + str(np.mean(ltsk)))
#print("Females are enrolled on average for a longer number of days, so might want to target males, but this is probably not a high return on investment because the distributions are not significantly different, and the effect on predicting churn is also small.")
stsk = studentsn[studentsn['shorter_term']==1]['age_enroll']
ltsk = studentsn[studentsn['shorter_term']==0]['age_enroll']
print("\nThe p value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).pvalue/float(2)) + " and the t value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).statistic/float(2)))
print ("The effect size is " + str(cohensd(stsk, ltsk)))
print("SHort terms enroll age " + str(np.mean(stsk)))
print("LOng terms enroll age " + str(np.mean(ltsk)))
#print("Females are enrolled on average for a longer number of days, so might want to target males, but this is probably not a high return on investment because the distributions are not significantly different, and the effect on predicting churn is also small.")
stsk = studentsn[studentsn['shorter_term']==1]['grade_level']
ltsk = studentsn[studentsn['shorter_term']==0]['grade_level']
print("\nThe p value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).pvalue/float(2)) + " and the t value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).statistic/float(2)))
print ("The effect size is " + str(cohensd(stsk, ltsk)))
print("SHort terms exit level " + str(np.mean(stsk)))
print("LOng terms exit level " + str(np.mean(ltsk)))
#print("Females are enrolled on average for a longer number of days, so might want to target males, but this is probably not a high return on investment because the distributions are not significantly different, and the effect on predicting churn is also small.")
stsk = studentsn[studentsn['shorter_term']==1]['days_open']
ltsk = studentsn[studentsn['shorter_term']==0]['days_open']
print("\nThe p value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).pvalue/float(2)) + " and the t value is " + str(sp.stats.ttest_ind(stsk, ltsk,axis=0, equal_var=False).statistic/float(2)))
print ("The effect size is " + str(cohensd(stsk, ltsk)))
print("SHort terms days open " + str(np.mean(stsk)))
print("LOng terms days open " + str(np.mean(ltsk)))

The p value is 2.14548071754e-18 and the t value is 4.4569254467
The effect size is 0.479008947353
SHort terms increase skills on average at rate 0.0279150399773
LOng terms increase skills on avergae at rate 0.00854169483269

 The p value is 0.00858941288064 and the t value is 1.19282269495
The effect size is 0.107652800726
SHort terms begin at grade level 6.206095791
LOng terms begin at grade level 5.94480519481

 The p value is 0.31307978416 and the t value is 0.243627735685
The effect size is 0.022434000145
SHort terms commute 0.0620986101336
LOng terms commute 0.0604933760781

The p value is 0.0131657293671 and the t value is -1.1118810653
The effect size is -0.101808846642
SHort terms income 0.409474417767
LOng terms income 0.422709115446

The p value is 5.74156454628e-09 and the t value is 2.87759537503
The effect size is 0.280345235092
SHort terms enroll age 9.39187227866
LOng terms enroll age 8.40714285714

The p value is 1.35382262906e-65 and the t value is -8.98095068665
The 

In [602]:
#studentsn.head(10)
# look at data from 2 angles
# winter / summer ?

,camp_id,student_id,gender,date_of_birth,enroll_date,end_date,grade_level,city_student,country_student,zip_student,...,age_2016,age_enroll,age_end,days_open,dist,household_income,median_income,grade_level_begin,skill_increase,shorter_term
0,2824,61913,M,2002-01-22,2015-09-01,2015-11-30,12,Santa Rosa,USA,95409,...,14,13,13,2,0.102496,0.298022,0.309560,11,0.008333,1
1,2824,61981,F,2008-01-21,2016-03-04,2016-05-28,7,Santa Rosa,USA,95409,...,8,8,8,2,0.102496,0.298022,0.309560,5,0.017647,1
2,2824,61991,F,2004-09-25,2016-03-04,2016-05-26,7,Santa Rosa,USA,95409,...,11,11,11,2,0.102496,0.298022,0.309560,6,0.009036,1
3,2824,62145,F,2005-06-11,2015-01-01,2015-08-23,10,Santa Rosa,USA,95409,...,11,9,10,2,0.102496,0.298022,0.309560,2,0.025641,0
4,2824,62186,F,2007-09-28,2015-05-01,2015-05-24,6,Santa Rosa,USA,95409,...,8,7,7,2,0.102496,0.298022,0.309560,5,0.032609,1
5,2824,62436,M,1999-09-09,2016-06-02,2016-09-08,9,Santa Rosa,USA,95409,...,17,16,17,2,0.102496,0.298022,0.309560,8,0.007653,1
6,2824,63963,M,2005-06-04,2015-05-04,2016-03-26,9,Santa Rosa,USA,95409,...,11,9,10,2,0.102496,0.298022,0.309560,5,0.009174,0
7,2825,477782,M,2006-01-04,2011-02-03,2013-06-30,8,Mill Valley,USA,95409,...,10,5,7,3,0.655610,0.298022,0.309560,2,0.005125,0
8,2825,478329,F,2009-03-18,2014-06-12,2015-12-21,7,Mill Valley,USA,95409,...,7,5,6,3,0.655610,0.298022,0.309560,2,0.006732,0
9,2824,61938,M,2000-12-13,2015-01-01,2015-08-30,12,Santa Rosa,USA,95404,...,15,14,14,2,0.048862,0.312023,0.328014,8,0.012448,0


In [ ]:
#

In [435]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn import linear_model, datasets, metrics
from sklearn.cross_validation import train_test_split

logreg = LogisticRegression(penalty = "l2", C=1, class_weight = 'balanced', tol = 0.0001)
#logreg = linear_model.LogisticRegression()
feature_cols = ['age_enroll','days_open', 'dist', 'median_income', 'grade_level']

xprev = studentsshortnew[feature_cols]#.reset_index()[feature_cols]
y = studentsshortnew['short_term']#.reset_index()['short_term']x
#x1 = []
#x1 = pd.concat([xprev, pd.get_dummies(studentsshortnew['gender'])],axis=1)
#x = []
#x = pd.concat([x1, pd.get_dummies(studentsshortnew['city_camp'])],axis=1)
#print x
x = xprev
#names = []
#names = perm_dict['city_camp'].keys()
#names.extend(perm_dict['gender'].keys())
#names.extend(perm_dict['days_open'].keys())
#names.extend(perm_dict['grade_level'].keys())
#names.extend(perm_dict['age_enroll'].keys())
#xprev.head()


X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2)
L = logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)

print 'Accuracy of model on test set is ' + str(metrics.accuracy_score(y_test,y_pred)*100) +' %'
print('F1 scores are long-term/short-term' + str(f1_score(y_test, y_pred, average=None)))
#print('Weighted F1 score is ' + str(f1_score(y_test, y_pred, average='weighted')))
print('Precision scores are lt-st' + str(precision_score(y_test, y_pred, average=None)))
#print('Weighted Precision score is ' + str(precision_score(y_test, y_pred, average='weighted')))
print('Recall scores are lt-st' + str(recall_score(y_test, y_pred, average=None)))
#print('Weighted Recall score is ' + str(recall_score(y_test, y_pred, average='weighted')))
print metrics.confusion_matrix(y_test, y_pred)
print ("Predicting everything as falling as long term will give accuracy of "+ str((len([i for i in y_test if i == 0])/float(len(y_test)))*100))

fimplr = logreg.coef_
vv = list(x)
#ll = np.argsort(np.abs(logreg.coef_))
#print(fimplr.ravel())
#print(vv)
#print(ll)
#print(ll[vv])


data = [Bar(x=vv,y=fimplr.ravel())]

layout = Layout(xaxis=dict(title = 'Features'),
                 yaxis=dict(title='Coefficients'),
                title='Feature Coefficients')


fig1 = Figure(data=data, layout=layout)
iplot(fig1)






# sort and plot positive and negative features on different plots
# pre process, normalize data
# combine gender as 1 feature? or remove gender?

# This is based on the idea that when all features are on the same scale, 
#the most important features should have the highest coefficients in the model, 

# normalize feature coefficients?
# engineer distance as a feature?
# correlated coefficients get similar features
# tail-end data, outliers

Accuracy of model on test set is 77.4049217002 %
F1 scores are long-term/short-term[ 0.77404922  0.77404922]
Precision scores are lt-st[ 0.75877193  0.78995434]
Recall scores are lt-st[ 0.78995434  0.75877193]
[[173  46]
 [ 55 173]]
Predicting everything as falling as long term will give accuracy of 48.9932885906


In [314]:
trace1 = Scatter(x=studentsshortnew['median_income'],y=studentsshortnew['time_enrolled'], mode='markers')
data = [trace1]
layout = Layout(xaxis=dict(title = 'Median Income'),
                 yaxis=dict(title='Time Enrolled'),
                title='Median Income vs Time Enrolled')


fig1 = Figure(data=data, layout=layout)
iplot(fig1)


In [437]:
len(studentsshortnew)

2233

In [240]:
trace1 = Scatter(x=studentsshortnew['dist'],y=studentsshortnew['time_enrolled'], mode='markers')
data = [trace1]
layout = Layout(xaxis=dict(title = 'Distance'),
                 yaxis=dict(title='Time Enrolled'),
                title='Distance vs Time Enrolled')


fig1 = Figure(data=data, layout=layout)
iplot(fig1)

#studentsshortnew['time_enrolled'].groupby(studentsshortnew['city_camp']).mean()
#print('Definitely time enrolled depends on city - Stockton, Los ALamitos, San Jose have high enrollment times, SF, San CLemente not great - IN LR not super clear whats going on')


city_camp
Concord          260.750000
Fairfield        156.322917
Fresno           165.155556
Los Alamitos     457.235294
Mission Viejo    292.266667
San Clemente      91.037037
San Francisco    149.254902
San Jose         448.547368
San Rafael       347.495601
Santa Clara      147.500000
Stockton         805.326531
Torrance         233.338028
Victorville      169.706522
Whittier         160.614583
Windsor          176.700000
Name: time_enrolled, dtype: float64

In [313]:

trace1 = Scatter(x=students['city_camp'],y=students['time_enrolled'], mode='markers')
data = [trace1]
layout = Layout(xaxis=dict(title = 'City of Camp'),
                 yaxis=dict(title='Time Enrolled'),
                title='City of Camp vs Time Enrolled')


fig1 = Figure(data=data, layout=layout)
iplot(fig1)

students['time_enrolled'].groupby(students['city_camp']).mean()
print('Definitely time enrolled depends on city - Stockton, Los ALamitos, San Jose have high enrollment times, SF, San CLemente not great - IN LR not super clear whats going on')


Definitely time enrolled depends on city - Stockton, Los ALamitos, San Jose have high enrollment times, SF, San CLemente not great - IN LR not super clear whats going on


In [562]:
male_time = students[students['gender'] == 'M']['time_enrolled']
female_time = students[students['gender'] == 'F']['time_enrolled']

import scipy as sp
def cohensd(val1,val2):
    return ( np.mean(val1) - np.mean(val2) ) / np.sqrt( (np.std(val1) ** 2 + np.std(val2) ** 2) / 2 )

print("The p value is " + str(sp.stats.ttest_ind(male_time, female_time,axis=0, equal_var=False).pvalue/float(2)) + " and the t value is " + str(sp.stats.ttest_ind(male_time, female_time,axis=0, equal_var=False).statistic/float(2)))
print ("The effect size is " + str(cohensd(male_time, female_time)))
print("Males are enrolled on average for " + str(int(np.mean(male_time))) + " days")
print("Females are enrolled on average for " + str(int(np.mean(female_time)))+ " days")
print("Females are enrolled on average for a longer number of days, so might want to target males, but this is probably not a high return on investment because the distributions are not significantly different, and the effect on predicting churn is also small.")

#iplot([Scatter(x=students['gender'],y=students['time_enrolled'], mode='markers')])
trace1 = Scatter(x=students['gender'],y=students['time_enrolled'], mode='markers')
data = [trace1]
layout = Layout(xaxis=dict(title = 'Gender'),
                 yaxis=dict(title='Time Enrolled'),
                title='Gender vs Time Enrolled')


fig1 = Figure(data=data, layout=layout)
iplot(fig1)


The p value is 0.0989691751991 and the t value is -0.643884990064
The effect size is -0.0499569449623
Males are enrolled on average for 373 days
Females are enrolled on average for 395 days
Females are enrolled on average for a longer number of days, so might want to target males, but this is probably not a high return on investment because the distributions are not significantly different, and the effect on predicting churn is also small.


In [314]:
twodays_time = students[students['days_open'] == 2]['time_enrolled']
threedays_time = students[students['days_open'] == 3]['time_enrolled']


print("The p value is " + str(sp.stats.ttest_ind(twodays_time, threedays_time,axis=0, equal_var=False).pvalue/float(2)) + " and the t value is " + str(sp.stats.ttest_ind(twodays_time, threedays_time,axis=0, equal_var=False).statistic/float(2)))
print ("The effect size is " + str(cohensd(twodays_time, threedays_time)))
print("Centers open for 2 days hav an average enrllment of " + str(int(np.mean(twodays_time))) + " days")
print("Centers open for 3 days hav an average enrllment of " + str(int(np.mean(threedays_time)))+ " days")
print(" The number of days the center is open does not seem to have a significant impact on the logistic regression - but the 2 distributions are significantly different: The centers open for 2 days have a higher mean enrollment time than those open for 3 days, albeit with a small effect size. This might be a feature we need to boost up in the LR model")
      
trace1 = Scatter(x=students['days_open'],y=students['time_enrolled'], mode='markers')
data = [trace1]
layout = Layout(xaxis=dict(title = 'Days Open'),
                 yaxis=dict(title='Time Enrolled'),
                title='Days Open vs Time Enrolled')


fig1 = Figure(data=data, layout=layout)
iplot(fig1)

The p value is 2.89972907965e-05 and the t value is 2.01488953574
The effect size is 0.160106934351
Centers open for 2 days hav an average enrllment of 402 days
Centers open for 3 days hav an average enrllment of 337 days
 The number of days the center is open does not seem to have a significant impact on the logistic regression - but the 2 distributions are significantly different: The centers open for 2 days have a higher mean enrollment time than those open for 3 days, albeit with a small effect size. This might be a feature we need to boost up in the LR model


In [277]:
#students['age_2016'] = (((ag_2016 / np.timedelta64(1, 'D')).astype(int))/365).astype(int)
#students['age_enroll'] = (((ag_enroll / np.timedelta64(1, 'D')).astype(int))/365).astype(int)
#students['age_end'] = (((ag_end / np.timedelta64(1, 'D')).astype(int))/365).astype(int)
ae = (((ag_enroll / np.timedelta64(1, 'D')).astype(int))/365).astype(int)
#ae

trace1 = Scatter(x=ae,y=students['time_enrolled'], mode='markers')
data = [trace1]
layout = Layout(xaxis=dict(title = 'Age of Student'),
                 yaxis=dict(title='Time Enrolled'),
                title='Age of student vs Time Enrolled')


fig1 = Figure(data=data, layout=layout)
iplot(fig1)

#iplot([Scatter(x=ae,y=students['time_enrolled'], mode='markers')])
print(np.corrcoef(ae,students['time_enrolled'])[0,1])
print('The age of enrollment definitely seems to have an impact on enrollment times - seems like higher age of enrollment means slightly lower enrollment times - seems like you might want to target people between 12 and 19 and 2 and 3 to enroll for longer periods; also beyond 1 and below 4 you have lower enrollment')
print('Logistic regression also seems to be picking this up - negative values for ages 2-12 and beyond 19, postive for btw 12 and 19')
# do some multi-variate statistics!!
students['time_enrolled'].groupby(ae).median()

-0.175321118586
The age of enrollment definitely seems to have an impact on enrollment times - seems like higher age of enrollment means slightly lower enrollment times - seems like you might want to target people between 12 and 19 and 2 and 3 to enroll for longer periods; also beyond 1 and below 4 you have lower enrollment
Logistic regression also seems to be picking this up - negative values for ages 2-12 and beyond 19, postive for btw 12 and 19


2     123.0
3     323.5
4     300.0
5     290.0
6     266.0
7     249.0
8     273.0
9     222.0
10    217.0
11    201.0
12    191.0
13    163.0
14    137.0
15    173.0
16     64.0
17     49.5
18    100.5
19     91.0
26    115.0
42     44.0
46     90.0
Name: time_enrolled, dtype: float64

In [285]:
trace1 = Scatter(x=grlv,y=students['time_enrolled'], mode='markers')
data = [trace1]
layout = Layout(xaxis=dict(title = 'Grade Level'),
                 yaxis=dict(title='Time Enrolled'),
                title='Grade Level vs Time Enrolled')


fig1 = Figure(data=data, layout=layout)
iplot(fig1)


print(np.corrcoef(grlv,students['time_enrolled'])[0,1])
print('Definitely grade level has a big impact on time of enrollment - higher level enrolled for longer time as shown by correlation cefficient, and data. ALso LR grades 19 and 16 have high impact but why is grade 3 showing up as high impact?')
students['time_enrolled'].groupby(grlv).mean()

0.512956988793
Definitely grade level has a big impact on time of enrollment - higher level enrolled for longer time as shown by correlation cefficient, and data. ALso LR grades 19 and 16 have high impact but why is grade 3 showing up as high impact?


grade_level
1       69.300000
2      102.304348
3       82.080000
4      125.425000
5      146.726496
6      177.319797
7      198.837638
8      238.309309
9      298.640704
10     344.505618
11     452.731884
12     493.546667
13     672.329268
14     759.294118
15     904.627907
16    1055.612903
17     898.409091
18     978.000000
19    1528.250000
20     918.666667
21    1585.400000
Name: time_enrolled, dtype: float64

In [265]:

#pd.get_dummies(students['city_camp'])

In [144]:
#perm_dict = {}
#perm_dict['city_camp'] = dict(zip( np.unique(x['city_camp']) ,range(len(np.unique(x['city_camp'])))))
#perm_dict['gender'] = dict(zip( np.unique(x['gender']) ,range(len(np.unique(x['gender'])))))
#perm_dict['days_open'] = dict(zip( np.unique(x['days_open']) ,range(len(np.unique(x['days_open'])))))
#perm_dict['grade_level'] = dict(zip( np.unique(x['grade_level']) ,range(len(np.unique(x['grade_level'])))))
#perm_dict['age_enroll'] = dict(zip( np.unique(x['age_enroll']) ,range(len(np.unique(x['age_enroll'])))))

#names = []
#names = perm_dict['city_camp'].keys()
#names.extend(perm_dict['gender'].keys())
#names.extend(perm_dict['days_open'].keys())
#names.extend(perm_dict['grade_level'].keys())
#names.extend(perm_dict['age_enroll'].keys())

#logreg = linear_model.LogisticRegression()
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2)
L = logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)

print 'Accuracy of model on test set is ' + str(metrics.accuracy_score(y_test,y_pred)*100) +' %'
#print('F1 scores are long-term/short-term' + str(f1_score(y_test, y_pred, average=None)))
#print('Weighted F1 score is ' + str(f1_score(y_test, y_pred, average='weighted')))
#print('Precision scores are lt-st' + str(precision_score(y_test, y_pred, average=None)))
#print('Weighted Precision score is ' + str(precision_score(y_test, y_pred, average='weighted')))
#print('Recall scores are lt-st' + str(recall_score(y_test, y_pred, average=None)))
#print('Weighted Recall score is ' + str(recall_score(y_test, y_pred, average='weighted')))
print metrics.confusion_matrix(y_test, y_pred)
print ("Predicting everything as falling as long term will give accuracy of "+ str((len([x for x in y_test if x == 0])/float(len(y_test)))*100))

fimplr = logreg.coef_
print(fimplr.ravel())
#print(names)


Accuracy of model on test set is 81.308411215 %
[[206  58]
 [ 42 229]]
Predicting everything as falling as long term will give accuracy of 49.3457943925
[ 0.68002403 -0.82071851  0.27077678  0.62284953  0.66152896 -0.96013697
  1.00692409  0.05497882 -1.41806359  0.22451581  6.38688541  0.14364624
 -0.89719126 -0.96185946  0.5212075  -1.56938937 -0.99966585 -0.19590317
  0.11803774 -0.16960746]


In [248]:
# LOGISTIC REGRESSION

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn import linear_model, datasets, metrics
from sklearn.cross_validation import train_test_split

students['grade_level'] = 'g' + students['grade_level'].astype(str)
students['age_enroll'] = 'a' + students['age_enroll'].astype(str)


#students['grade_level'] = students['grade_level'] - 23
#students['age'] = students['age'] + 50

feature_cols = ['city_camp', 'gender', 'days_open', 'grade_level', 'age_enroll']
x = students[feature_cols].reset_index()[feature_cols]
y = students['short_term'].reset_index()['short_term']

In [249]:
perm_dict = {}
perm_dict['city_camp'] = dict(zip( np.unique(x['city_camp']) ,range(len(np.unique(x['city_camp'])))))
perm_dict['gender'] = dict(zip( np.unique(x['gender']) ,range(len(np.unique(x['gender'])))))
perm_dict['days_open'] = dict(zip( np.unique(x['days_open']) ,range(len(np.unique(x['days_open'])))))
perm_dict['grade_level'] = dict(zip( np.unique(x['grade_level']) ,range(len(np.unique(x['grade_level'])))))
perm_dict['age_enroll'] = dict(zip( np.unique(x['age_enroll']) ,range(len(np.unique(x['age_enroll'])))))

names = []
names = perm_dict['city_camp'].keys()
names.extend(perm_dict['gender'].keys())
names.extend(perm_dict['days_open'].keys())
names.extend(perm_dict['grade_level'].keys())
names.extend(perm_dict['age_enroll'].keys())

x = x.replace(to_replace = perm_dict)
enc = OneHotEncoder()
x2= enc.fit_transform(x)
xnew = x2.todense()
xnew
names

['San Clemente',
 'San Jose',
 'Victorville',
 'Santa Clara',
 'Mission Viejo',
 'Fresno',
 'Concord',
 'Fairfield',
 'Los Alamitos',
 'Whittier',
 'San Francisco',
 'San Rafael',
 'Windsor',
 'Torrance',
 'Stockton',
 'M',
 'F',
 2,
 3,
 'gg11',
 'gg10',
 'gg13',
 'gg12',
 'gg1',
 'gg14',
 'gg17',
 'gg16',
 'gg19',
 'gg18',
 'gg5',
 'gg2',
 'gg9',
 'gg7',
 'gg8',
 'gg6',
 'gg21',
 'gg20',
 'gg15',
 'gg4',
 'gg3',
 'aa42',
 'aa5',
 'aa6',
 'aa46',
 'aa3',
 'aa4',
 'aa2',
 'aa7',
 'aa26',
 'aa11',
 'aa10',
 'aa13',
 'aa12',
 'aa15',
 'aa14',
 'aa17',
 'aa16',
 'aa19',
 'aa18',
 'aa9',
 'aa8']

In [245]:
#logreg = linear_model.LogisticRegression()
logreg = LogisticRegression(penalty = "l2", C=0.1, class_weight = 'balanced', tol = 0.0001)
X_train,X_test,y_train,y_test = train_test_split(xnew,y,test_size=0.2)
L = logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)

print 'Accuracy of model on test set is ' + str(metrics.accuracy_score(y_test,y_pred)*100) +' %'
print('F1 scores are long-term/short-term' + str(f1_score(y_test, y_pred, average=None)))
print('Weighted F1 score is ' + str(f1_score(y_test, y_pred, average='weighted')))
print('Precision scores are lt-st' + str(precision_score(y_test, y_pred, average=None)))
print('Weighted Precision score is ' + str(precision_score(y_test, y_pred, average='weighted')))
print('Recall scores are lt-st' + str(recall_score(y_test, y_pred, average=None)))
print('Weighted Recall score is ' + str(recall_score(y_test, y_pred, average='weighted')))
print metrics.confusion_matrix(y_test, y_pred)
print ("Predicting everything as falling as long term will give accuracy of "+ str((len([x for x in y_test if x == 0])/float(len(y_test)))*100))

fimplr = logreg.coef_
print(fimplr.ravel())
print(names)

data = [Bar(x=names,y=fimplr.ravel())]

layout = Layout(xaxis=dict(title = 'Features'),
                 yaxis=dict(title='Coefficients'),
                title='Feature Coefficients')


fig1 = Figure(data=data, layout=layout)
iplot(fig1)

#iplot(data, filename='basic-bar')

Accuracy of model on test set is 76.6355140187 %
F1 scores are long-term/short-term[ 0.7637051  0.7689464]
Weighted F1 score is 0.766340444977
Precision scores are lt-st[ 0.76806084  0.76470588]
Weighted Precision score is 0.766373953014
Recall scores are lt-st[ 0.7593985  0.7732342]
Weighted Recall score is 0.766355140187
[[202  64]
 [ 61 208]]
Predicting everything as falling as long term will give accuracy of 49.7196261682
[ 0.0162543   0.56340544  0.13292464 -0.71407172  0.23363453  0.54921249
  0.18923651 -0.21253443 -0.25229858  0.45412122 -1.20022743 -0.20658825
  0.19183157  0.20211021  0.21173826  0.04815862  0.11059014  0.15320625
  0.00554252  0.37864796 -0.12665303 -0.65743237 -0.77547528 -1.09381524
 -1.01294882 -0.97173963 -1.05260729 -0.48174335 -0.26474687 -0.10456682
  0.80041191  0.         -0.09644692  0.88777021  0.67295925  1.3048633
  1.05761196  1.09713887  0.56304924  0.03447169  0.36642116  0.5092812
  0.43688148  0.92439993  0.75005128  0.22600845  0.23779577 

In [17]:
logreg = linear_model.LogisticRegression()
X_train,X_test,y_train,y_test = train_test_split(xnew,y,test_size=0.2)
L = logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)

print 'Accuracy of model on test set is ' + str(metrics.accuracy_score(y_test,y_pred)*100) +' %'
print('F1 scores are long-term/short-term' + str(f1_score(y_test, y_pred, average=None)))
print('Weighted F1 score is ' + str(f1_score(y_test, y_pred, average='weighted')))
print('Precision scores are lt-st' + str(precision_score(y_test, y_pred, average=None)))
print('Weighted Precision score is ' + str(precision_score(y_test, y_pred, average='weighted')))
print('Recall scores are lt-st' + str(recall_score(y_test, y_pred, average=None)))
print('Weighted Recall score is ' + str(recall_score(y_test, y_pred, average='weighted')))
print metrics.confusion_matrix(y_test, y_pred)
print ("Predicting everything as falling as long term will give accuracy of "+ str((len([x for x in y_test if x == 0])/float(len(y_test)))*100))

fimplr = logreg.coef_
print(fimplr.ravel())
print(names)

data = [Bar(x=names,y=fimplr.ravel())]

layout = Layout(xaxis=dict(title = 'Features'),
                 yaxis=dict(title='Coefficients'),
                title='Feature Coefficients')


fig1 = Figure(data=data, layout=layout)
iplot(fig1)

#iplot(data, filename='basic-bar')


Accuracy of model on test set is 81.1214953271 %
F1 scores are long-term/short-term[ 0.81399632  0.80834915]
Weighted F1 score is 0.811251897378
Precision scores are lt-st[ 0.82462687  0.79775281]
Weighted Precision score is 0.811566576443
Recall scores are lt-st[ 0.80363636  0.81923077]
Weighted Recall score is 0.811214953271
[[221  54]
 [ 47 213]]
Predicting everything as falling as long term will give accuracy of 51.4018691589
[ -1.21238633e-01   8.60788393e-01   4.25369642e-01  -1.07540539e+00
   1.84259857e-01   1.65626411e+00   2.44895644e-01  -2.67976240e-01
  -4.10670729e-01   8.94530950e-01  -1.35261022e+00  -6.17555704e-01
   5.67107724e-03   1.00247908e-01  -9.48739469e-02   1.61801946e-01
   2.69894767e-01   3.37051365e-01   9.46453480e-02   3.39447249e+00
  -6.79088413e-01  -1.31123929e+00  -1.86549379e+00  -2.23441837e+00
  -2.26612831e+00  -2.77838139e+00  -2.90957223e+00  -2.41377300e+00
  -1.99432117e+00  -9.92351451e-01   3.41633619e+00  -8.25551704e-01
  -7.96990898e

In [23]:
# TO - DOS

# Feature engineering - distance from house, grade improvements, population / education level of areas 
# improve prediction by logisti regression, dont bother with random forests, 
# no of students per camp, average gade per cap - LATER POST DEMO

# more data - LATER POST DEMO

# improve logistic regression - POST DEMO

# draw a cumulative plot to justify or run it by michael or do 50/50 - easy ROI

# scaling issues with coefficients and normalization - should do this - ROI

# regularization - this

# plot for those who stayed vs not stayed - the  2 vs 3 days open - this


# story with business idea, background research n education and students and how it translates to companies

# Mainly : Prep Demos, Talk to Michael

# 10 - 12 - just do demo prep and slides

# 3 - 6
# SLIDES AND WHAT TO THINK ABOUT LR - main goal for today

In [434]:

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=10)

feature_cols = ['age_enroll','days_open', 'dist', 'median_income', 'grade_level']

xprev = studentsshortnew[feature_cols]#.reset_index()[feature_cols]
y = studentsshortnew['short_term']#.reset_index()['short_term']x

x = xprev
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2)
L = rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)

print 'Accuracy of model on test set is ' + str(metrics.accuracy_score(y_test,y_pred)*100) +' %'
#print('F1 scores are long-term/short-term' + str(f1_score(y_test, y_pred, average=None)))
#print('Weighted F1 score is ' + str(f1_score(y_test, y_pred, average='weighted')))
#print('Precision scores are lt-st' + str(precision_score(y_test, y_pred, average=None)))
#print('Weighted Precision score is ' + str(precision_score(y_test, y_pred, average='weighted')))
#print('Recall scores are lt-st' + str(recall_score(y_test, y_pred, average=None)))
#print('Weighted Recall score is ' + str(recall_score(y_test, y_pred, average='weighted')))
print metrics.confusion_matrix(y_test, y_pred)
print ("Predicting everything as falling as long term will give accuracy of "+ str((len([x for x in y_test if x == 0])/float(len(y_test)))*100))

fimplr = rf.feature_importances_
print(fimplr.ravel())
print(vv)

Accuracy of model on test set is 77.4049217002 %
[[160  44]
 [ 57 186]]
Predicting everything as falling as long term will give accuracy of 45.6375838926
[ 0.26590629  0.02293586  0.13610383  0.20665322  0.36840079]
['age_enroll', 'days_open', 'dist', 'median_income', 'grade_level']


In [164]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=10)
feature_cols = ['age_enroll', 'grade_level', 'days_open']

xprev = students[feature_cols]#.reset_index()[feature_cols]
y = students['short_term']#.reset_index()['short_term']x
x1 = []
x1 = pd.concat([xprev, pd.get_dummies(students['gender'])],axis=1)
x = []
x = pd.concat([x1, pd.get_dummies(students['city_camp'])],axis=1)


#names = []
#names = perm_dict['city_camp'].keys()
#names.extend(perm_dict['gender'].keys())
#names.extend(perm_dict['days_open'].keys())
#names.extend(perm_dict['grade_level'].keys())
#names.extend(perm_dict['age_enroll'].keys())


X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2)
L = rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)

print 'Accuracy of model on test set is ' + str(metrics.accuracy_score(y_test,y_pred)*100) +' %'
#print('F1 scores are long-term/short-term' + str(f1_score(y_test, y_pred, average=None)))
#print('Weighted F1 score is ' + str(f1_score(y_test, y_pred, average='weighted')))
#print('Precision scores are lt-st' + str(precision_score(y_test, y_pred, average=None)))
#print('Weighted Precision score is ' + str(precision_score(y_test, y_pred, average='weighted')))
#print('Recall scores are lt-st' + str(recall_score(y_test, y_pred, average=None)))
#print('Weighted Recall score is ' + str(recall_score(y_test, y_pred, average='weighted')))
print metrics.confusion_matrix(y_test, y_pred)
print ("Predicting everything as falling as long term will give accuracy of "+ str((len([x for x in y_test if x == 0])/float(len(y_test)))*100))

fimplr = rf.feature_importances_
print(fimplr.ravel())
#print(names)

Accuracy of model on test set is 76.4485981308 %
[[210  66]
 [ 60 199]]
Predicting everything as falling as long term will give accuracy of 51.5887850467
[ 0.33678739  0.41748808  0.0120963   0.01748153  0.01718691  0.00778378
  0.01666949  0.00700612  0.02199968  0.01325707  0.00609393  0.00780081
  0.00875785  0.00902257  0.00948882  0.04779809  0.01182748  0.01302717
  0.01059545  0.00783148]


In [24]:
# RANDOM FORESTS

from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier(n_estimators=10)
X_train,X_test,y_train,y_test = train_test_split(xnew,y,test_size=0.2)

rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print 'Accuracy of model on test set is ' + str(metrics.accuracy_score(y_test,y_pred)*100) +' %' + '\n'
print metrics.confusion_matrix(y_test, y_pred)
print ("Predicting everything as falling as long term will still give accuracy of "+ str((len([x for x in y_test if x == 0])/float(len(y_test)))*100))


print(precision_score(y_test, y_pred, average=None))
#print(recall_score(y_test, y_pred, average=None))  
#print('F1 scores are' + str(f1_score(y_test, y_pred, average=None)))
print('Weighted F1 score is ' + str(f1_score(y_test, y_pred, average='weighted')))
#print('Precision scores are' + str(precision_score(y_test, y_pred, average=None)))
print('Weighted Precision score is ' + str(precision_score(y_test, y_pred, average='weighted')))
#print('Recall scores are' + str(recall_score(y_test, y_pred, average=None)))
print('Weighted Recall score is ' + str(recall_score(y_test, y_pred, average='weighted'))) 

Accuracy of model on test set is 77.3831775701 %

[[204  66]
 [ 55 210]]
Predicting everything as falling as long term will still give accuracy of 50.4672897196
[ 0.78764479  0.76086957]
Weighted F1 score is 0.773779617502
Weighted Precision score is 0.774382294293
Weighted Recall score is 0.773831775701


In [25]:
fimp = sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_),names),reverse=True)
print(fimp[:10])
trace0 = Bar(x=[x[1] for x in fimp[:10]],y=[x[0] for x in fimp[:10]],marker=dict(color='rgb(26, 118, 255)',line=dict(color='rgb(8,48,107)',width=1,)),opacity=0.7)
data = [trace0]
layout = Layout(xaxis=dict(title = 'Features', tickangle=-45, tickfont=dict(size=12,color='rgb(107, 107, 107)')),
                 yaxis=dict(title='Mean Decrease Impurity Score',titlefont=dict(size=16,color='rgb(107, 107, 107)'), tickfont=dict(size=14,color='rgb(107, 107, 107)')),
                title='Feature Importance of Top 20 Features',
                barmode='group',bargap=0.4,bargroupgap=0.1)
fig16 = Figure(data=data, layout=layout)
iplot(fig16, filename='feature-importance-mean-impurity-bar')


[(0.0519, 'San Francisco'), (0.0404, 'g11'), (0.0385, 'g12'), (0.0349, 'g10'), (0.0338, 'F'), (0.0321, 'a7'), (0.031, 'Santa Clara'), (0.0308, 'a6'), (0.0291, 'g6'), (0.0284, 'a8')]


In [26]:
students['enroll_date'].groupby(students['enroll_date'].dt.year).size().values

trace0 = Scatter(x = [2007,2008,2009,2010,2011,2012,2013,2014,2015,2016],y = students['enroll_date'].groupby(students['enroll_date'].dt.year).size().values,
                    mode = 'lines+markers',name = 'Number of students enrolled')
trace1 = Scatter(x = [2007,2008,2009,2010,2011,2012,2013,2014,2015], y = camps.groupby(camps[camps['state_code']== 'CA']['open_date'].dt.year).size().values[9:18],
                 mode = 'lines+markers',name = 'Number of camps opened')
trace2 = Scatter(x = [2007,2008,2009,2010,2011,2012,2013,2014,2015, 2016], y = students['end_date'].groupby(students['end_date'].dt.year).size().values,
                 mode = 'lines+markers',name = 'Number of students who quit')
data = [trace0, trace1, trace2]
layout = dict(title = 'Trends over the years')
fig = dict(data=data, layout=layout)
iplot(fig, filename='scatter-mode')




In [27]:
trace1 = Scatter(x=students['city_camp'],y=students['time_enrolled'], mode='markers')
trace2 = Scatter(x=students['grade_level'],y=students['time_enrolled'], mode='markers')
trace3 = Scatter(x=students['age'],y=students['time_enrolled'], mode='markers')
trace4 = Scatter(x=students['gender'],y=students['time_enrolled'], mode='markers')

fig = tools.make_subplots(rows=2, cols=2, subplot_titles=('City-Time', 'Grade-Time','Age-Time', 'Gender-Time'))
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)

fig['layout'].update(height=600, width=1000, title='Subplots of Dependent vs Independent Variables')
#iplot(fig, filename='make-subplots-multiple-with-title')


KeyError: 'age'

In [ ]:
# SOME PLOTS

#students.head()
from plotly import tools

trace1 = Scatter(x=students['city_camp'],y=students['time_enrolled'], mode='markers')
trace2 = Scatter(x=students['grade_level'],y=students['time_enrolled'], mode='markers')
trace3 = Scatter(x=students['age'],y=students['time_enrolled'], mode='markers')
trace4 = Scatter(x=students['gender'],y=students['time_enrolled'], mode='markers')

fig = tools.make_subplots(rows=2, cols=2, subplot_titles=('City-Time', 'Grade-Time','Age-Time', 'Gender-Time'))
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)

fig['layout'].update(height=600, width=1000, title='Subplots of Dependent vs Independent Variables')
#iplot(fig, filename='make-subplots-multiple-with-title')

#data = [trace]
#iplot(data, filename='basic-scatter')
#from plotly.tools import FigureFactory as FF
#st1 = students[['city_camp', 'grade_level']]#, 'age', 'gender']]
#fig = FF.create_scatterplotmatrix(st1, diag = 'histogram', height=800, width=800)
#iplot(fig, filename='Basic Scatterplot Matrix')

In [ ]:
trace1 = Scatter(x=students['city_camp'],y=students['grade_level'], mode='markers')
trace2 = Scatter(x=students['city_camp'],y=students['age'], mode='markers')
trace3 = Scatter(x=students['city_camp'],y=students['gender'], mode='markers')
trace4 = Scatter(x=students['grade_level'],y=students['age'], mode='markers')
trace5 = Scatter(x=students['grade_level'],y=students['gender'], mode='markers')
trace6 = Scatter(x=students['gender'],y=students['age'], mode='markers')

fig = tools.make_subplots(rows=3, cols=2, subplot_titles=('City-Grade', 'City-Age','City-Gender', 'Grade-Age', 'Grade-Gender','Gender-Age'))
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)
fig.append_trace(trace5, 3, 1)
fig.append_trace(trace6, 3, 2)

fig['layout'].update(height=900, width=1000, title='Subplots of Independent Variables')
#iplot(fig, filename='correlation matrix')


In [ ]:
#a = np.array([students['city_camp'], students['grade_level'], students['gender'], students['age']]) 
#b = np.corrcoef(a)

In [ ]:
# plot feature vs response

# plot feature vs feature

# try linear regression?

# understand what coefficients mean?

In [ ]:
# LINEAR REGRESSION, VERY POOR - PROBABLY NEED MORE DATA

y = students['time_enrolled'].reset_index()['time_enrolled']


from sklearn import datasets, metrics
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
lm = LinearRegression()
x_train,x_test,y_train,y_test = train_test_split(xnew, y,test_size=0.2)

lm.fit(x_train, y_train)

print("Sum of squares error: %.2f"
      % np.mean((lm.predict(x_test) - y_test) ** 2))
print('R squared value: %.2f' % lm.score(x_test, y_test))
print("Median of test data: %.2f" % np.mean(y_test))
print("Median of prediction: %.2f" % np.median(lm.predict(x_test)))
print("Std dev of test data: %.2f" % np.std(y_test))
print("Std dev of prediction: %.2f" % np.std(lm.predict(x_test)))


In [ ]:
students.groupby('camp_id').index.get_level_values('end_date')
#['end_date']#.size()
students.end_date.max()

#assume last day of worksheet submission is last day



#t = students[students['end_date'] != students.end_date.max()]['time_enrolled']

trace1 = Histogram(x=t,opacity=0.75)
trace2 = Histogram(x=students['time_enrolled'],opacity=0.75)
data1 = [trace1, trace2]
layout1 = Layout(barmode='overlay')
fig1 = Figure(data=data1, layout=layout1)
iplot(fig1)

# 2 distributions look somewhat similar, should be good for now but not sure..

students = students[students['end_date'] <= students.end_date.max()-timedelta(days=7)]

#students['city'] = students['city'].str.upper()
#students.groupby('city').size()

#clean cities






#students.head()


students[students['camp_city'] == 'San Francisco']

# students who quit within 3-4 months vs those who stayed longer.. ~1000 points

# or students who quit within a year

#are students bound to stay if their grade increases

# are they bound to stay based on their neighborhood

# should i do by state or restrict to CA

Improvements:
    
    Add scores
    #scores = pd.read_csv('ScoresCA.csv')

In [ ]:
# USER SEGMENT AND 2 lr - better models and telling features

# clean up city they are from - 150 cities - cluster around neighborhoods

# RF for feature importance

# decision trees

# toddler / teen / 
# importance of features as a whole